# Proyectos

# 1. Detección de patrones lingïsticos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import re

**Texto:** El día estaba soleado y la gente caminaba despacio por la plaza.
Los niños jugaban cerca de la fuente, mientras los adultos conversaban tranquilamente. Todo parecía en calma.

In [ ]:
texto ="El día estaba soleado y la gente caminaba despacio por la plaza. Los niños jugaban cerca de la fuente, mientras los adultos conversaban tranquilamente. Todo parecía en calma."

Lista de palabras (limpias, sin comas ni puntos):

In [ ]:
palabras = re.findall(r'\b\w+\b', texto.lower()) # Convierte todo el contenido de la variable texto a minúsculas.\b significa límite de palabra (inicio o fin).
                                                 #\w+ significa una o más letras, números o guiones bajos (caracteres alfanuméricos).
palabras

In [ ]:
len(palabras) #Tamaño de palabras, 28 palabras

In [ ]:
# Tabla de datos
df = pd.DataFrame({
    "palabra": palabras,
    "longitud": [len(p) for p in palabras]
})

print(df.head())


Texto es información cualitativa, para analizalo cuantitativamente necesitamos transformar el texto en números.

La longitud de cada palabra (en caracteres) es una de las formas más comunes y sencillas de convertir un texto en datos numéricos.

La longitud es solo un ejemplo sencillo.

Sobre texto, puedes crear muchas variables numéricas, por ejemplo:

✔ Número de palabras por oración

✔ Número de verbos

✔ Número de sustantivos

✔ Palabras por frase

✔ Frecuencia de una palabra

✔ Porcentaje de uso de una categoría (adjetivos, artículos, etc.)

✔ Puntaje de sentimiento

✔ Cantidad de signos de puntuación

✔ Complejidad léxica

✔ Diversidad léxica (relación entre palabras únicas / totales)


Todas estas pueden convertirse en números y permitir estadísticas descriptivas.

In [ ]:
# Estadística descriptiva
print("\nEstadística descriptiva:")
print(df["longitud"].describe())

In [ ]:
#Frecuencia de palabras
frecuencias = Counter(palabras)
df_freq = pd.DataFrame(frecuencias.items(), columns=["palabra", "frecuencia"])
print(df_freq.sort_values("frecuencia", ascending=False))

In [ ]:
#Gráficas
plt.hist(df["longitud"])
plt.xlabel("Longitud de palabra")
plt.ylabel("Frecuencia")
plt.title("Distribución de longitudes")
plt.show()

In [ ]:
df_freq.sort_values("frecuencia", ascending=False).head(10).plot(
    x="palabra", y="frecuencia", kind="bar"
)
plt.title("Palabras más frecuentes")
plt.show()

# 2 “Análisis de datos y detección de fallos operativos en una torre empacada de absorción aire–CO₂ mediante series temporales”

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
np.random.seed(42)
#Generar señal sintética
# --- 1. Segmento normal ---
n1 = 500
normal = np.random.normal(loc=0.08, scale=0.02, size=n1)

# --- 2. Segmento pre-flooding ---
n2 = 300
pref = np.random.normal(loc=0.12, scale=0.06, size=n2)

# --- 3. Segmento flooding ---
n3 = 300
flood = np.random.normal(loc=0.25, scale=0.15, size=n3)

# Añadir picos fuertes (outliers)
outliers = np.random.choice(n3, size=20, replace=False)
flood[outliers] += np.random.uniform(0.3, 0.8, size=20)

# Asegurar que no haya negativos
flood = np.clip(flood, 0, None)
pref = np.clip(pref, 0, None)
normal = np.clip(normal, 0, None)

# --- 4. Segmento recuperación ---
n4 = 250
rec = np.random.normal(loc=0.1, scale=0.03, size=n4)
rec = np.clip(rec, 0, None)

# Unir todo
co2 = np.concatenate([normal, pref, flood, rec])

# Crear tiempos sintéticos
time = pd.date_range(start="2025-10-15 10:00", periods=len(co2), freq="S")  # cada segundo

df = pd.DataFrame({
    "Time": time,
    "CO2": co2
})

print(df.head())
print(df.tail())



In [ ]:
df.to_csv("datos_sinteticos_CO2.csv", index=False)


In [ ]:
plt.figure(figsize=(14,4))
plt.plot(df["CO2"])
plt.title("Datos sintéticos de concentración de CO₂ en una torre empacada")
plt.xlabel("s")
plt.ylabel("CO₂")
plt.show()


In [ ]:
from google.colab import files
#files.download("datos_sinteticos_CO2.csv")



# 2

In [ ]:
#Cargar los datos
ruta="https://raw.githubusercontent.com/marinlopabi/Temas-Selectos-PQ-AD-Python/refs/heads/main/Datos/datos_sinteticos_CO2.csv"
df = pd.read_csv(ruta, parse_dates=["Time"])
df.head()

In [ ]:
#serie
plt.figure(figsize=(14,4))
plt.plot(df["Time"], df["CO2"], linewidth=1)
plt.title("Serie temporal de CO₂")
plt.xlabel("Tiempo")
plt.ylabel("Concentración CO₂")
plt.show()


In [ ]:
#Resumen estadístico
df["CO2"].describe()


In [ ]:
#histograma
plt.hist(df["CO2"], bins=50)
plt.title("Histograma de concentración de CO₂")
plt.xlabel("CO₂ (%)")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
#Gráfico de cajas
plt.boxplot(df["CO2"], vert=False)
plt.title("Boxplot de CO₂")
plt.xlabel("CO₂ (%)")
plt.show()


In [ ]:
#Tendencia
df["rolling_mean"] = df["CO2"].rolling(window=60).mean()
df["rolling_std"]  = df["CO2"].rolling(window=60).std()

plt.figure(figsize=(14,4))
plt.plot(df["Time"], df["rolling_mean"], label="Media móvil (60)")
plt.plot(df["Time"], df["rolling_std"], label="Desviación móvil (60)")
plt.legend()
plt.title("Media y desviación estándar móviles")
plt.xlabel("Tiempo")
plt.show()


“El análisis exploratorio sirve para entender el comportamiento de la serie.
En un proceso real, cuando la torre se inunda, la señal de CO₂ se vuelve caótica: aparecen picos, aumenta la desviación estándar y la distribución se deforma.
El EDA nos permite identificar esos cambios sin necesidad de modelos complicados.”

Segmentación: La idea es dividir la serie en zonas operativas:

Antes del flooding → operación normal

Transición / pre-flooding

Flooding (inundación)

Recuperación

Esto es muy común en ingeniería de procesos: separar un proceso continuo en regímenes dinámicos para entender qué está pasando.

# Segmentación de la serie temporal

Como estamos usando datos sintéticos, ya sabemos aproximadamente dónde está el flooding:

Segmento normal: primeros 500 datos

Pre-flooding: 500 a 800

Flooding: 800 a 1100

Recuperación: 1100 al final

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,4))
plt.plot(df["CO2"], label="CO2")

plt.axvline(500, color='green', linestyle='--', label="Fin normal")
plt.axvline(800, color='orange', linestyle='--', label="Inicio flooding")
plt.axvline(1100, color='red', linestyle='--', label="Fin flooding")

plt.title("Segmentación de la serie temporal de CO₂")
plt.xlabel("Índice")
plt.ylabel("CO₂ (%)")
plt.legend()
plt.show()


Se aprecian claramente cuatro zonas.

La variabilidad cambia radicalmente en flooding.

Las líneas verticales ayudan a visualizar transiciones de régimen.

In [ ]:
#Crear las cuatro secciones

normal = df.iloc[:500]
preflood = df.iloc[500:800]
flood = df.iloc[800:1100]
recover = df.iloc[1100:]


In [ ]:
#Resumen estadístico por segmento
print("Normal:\n", normal["CO2"].describe())
print("\nPre-flooding:\n", preflood["CO2"].describe())
print("\nFlooding:\n", flood["CO2"].describe())
print("\nRecuperación:\n", recover["CO2"].describe())


In [ ]:
#Visualizar cada segmento
fig, ax = plt.subplots(4,1, figsize=(12,10), sharex=True)

ax[0].plot(normal["CO2"], color="green")
ax[0].set_title("Normal")

ax[1].plot(preflood["CO2"], color="orange")
ax[1].set_title("Pre-flooding")

ax[2].plot(flood["CO2"], color="red")
ax[2].set_title("Flooding")

ax[3].plot(recover["CO2"], color="blue")
ax[3].set_title("Recuperación")

plt.tight_layout()
plt.show()


In [ ]:
#Estadísticas comparativas entre segmentos
def resumen(segmento):
    return pd.Series({
        "media": segmento["CO2"].mean(),
        "mediana": segmento["CO2"].median(),
        "std": segmento["CO2"].std(),
        "min": segmento["CO2"].min(),
        "max": segmento["CO2"].max(),
        "IQR": segmento["CO2"].quantile(0.75) - segmento["CO2"].quantile(0.25),
        "coef_var": segmento["CO2"].std() / segmento["CO2"].mean()
    })

stats = pd.DataFrame({
    "Normal": resumen(normal),
    "Pre-flooding": resumen(preflood),
    "Flooding": resumen(flood),
    "Recuperación": resumen(recover)
})

stats


In [ ]:
#Gráfico de cajas por segmento
plt.figure(figsize=(10,6))
plt.boxplot([normal["CO2"], preflood["CO2"], flood["CO2"], recover["CO2"]],
            labels=["Normal", "Pre-flooding", "Flooding", "Recuperación"])
plt.ylabel("CO₂ (%)")
plt.title("Comparación estadística entre segmentos")
plt.show()


Conclusión: un proceso químico deja huellas estadísticas cuando cambia de régimen:

Más ruido

Más dispersión

Más picos

Más variabilidad

- Esto es precisamente lo que ocurre cuando una torre empacada se inunda.

# Autocorrrelación


¿Qué es la autocorrelación?

Es una medida de qué tanto un dato se parece a otro que ocurrió antes.

Si la torre opera normal → los datos cambian lentamente → alta autocorrelación.

Si hay flooding → el proceso se vuelve caótico → la autocorrelación baja.
 En ingeniería, la autocorrelación revela estabilidad o inestabilidad de un proceso.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12,4))
plot_acf(normal["CO2"], lags=40)
plt.title("ACF - Operación normal")
plt.show()

plt.figure(figsize=(12,4))
plot_pacf(normal["CO2"], lags=40, method='ywm')
plt.title("PACF - Operación normal")
plt.show()


In [ ]:
plt.figure(figsize=(12,4))
plot_acf(flood["CO2"], lags=40)
plt.title("ACF - Flooding")
plt.show()

plt.figure(figsize=(12,4))
plot_pacf(flood["CO2"], lags=40, method='ywm')
plt.title("PACF - Flooding")
plt.show()


# 3. Análisis multivariable de contaminantes

In [ ]:
ruta="https://raw.githubusercontent.com/marinlopabi/Temas-Selectos-PQ-AD-Python/refs/heads/main/Datos/contam2022.csv"

In [ ]:
df=pd.read_csv(ruta, parse_dates = True, index_col=0)
df.head()

In [ ]:
df.columns = ['ozono','dioxido_azufre',
                  'dioxido_nitrogeno','monoxido_carbono',
                  'PM10'] #Renombrar las columnas de tu DataFrame llamado

In [ ]:
df.head()

In [ ]:
#  Inspección básica

print("Columnas:\n", df.columns, "\n")
print(df.info())
df.describe()


# 4. Secado de manzana

In [ ]:
!pip install scipy --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


In [ ]:
# Tiempos en minutos
tiempo = np.array([0, 10, 20, 30, 45, 60, 75, 90, 120, 150, 180])

# Masa húmeda de la muestra (g) - DATOS SINTÉTICOS
masa = np.array([100, 92, 85, 79, 72, 68, 64, 61, 57, 54, 52])

df = pd.DataFrame({
    "tiempo_min": tiempo,
    "masa_g": masa
})

df


humedad base seca y MR

In [ ]:
masa_seca = 50.0  # g, asumida

# masa de agua en cada tiempo
masa_agua = df["masa_g"] - masa_seca

# Humedad base seca: X = (masa de agua) / (masa seca)
df["X_bs"] = masa_agua / masa_seca

# Humedad inicial y (suponemos) humedad de equilibrio Xe ~ 0
X0 = df["X_bs"].iloc[0]
Xe = 0.0

# Moisture Ratio MR = (X - Xe) / (X0 - Xe) ≈ X / X0
df["MR"] = df["X_bs"] / X0

df


In [ ]:
#Curva de secado
plt.figure(figsize=(6,4))
plt.plot(df["tiempo_min"], df["X_bs"], marker="o")
plt.xlabel("Tiempo (min)")
plt.ylabel("Humedad X (base seca)")
plt.title("Curva de secado (X vs tiempo)")
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(df["tiempo_min"], df["MR"], marker="o")
plt.xlabel("Tiempo (min)")
plt.ylabel("Moisture Ratio (MR)")
plt.title("Curva de secado (MR vs tiempo)")
plt.grid(True)
plt.show()


In [ ]:
#Velocidad de secado
# Diferencias entre puntos
dX = np.diff(df["X_bs"].values)
dt = np.diff(df["tiempo_min"].values)

#  secado (positiva)
DR = -dX / dt  # signo menos porque X disminuye

# Definimos el tiempo intermedio para graficar
t_mid = (df["tiempo_min"].values[:-1] + df["tiempo_min"].values[1:]) / 2

df_DR = pd.DataFrame({
    "tiempo_mid": t_mid,
    "DR": DR
})

df_DR


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(df_DR["tiempo_mid"], df_DR["DR"], marker="o")
plt.xlabel("Tiempo (min)")
plt.ylabel("Razón de secado DR (1/min)")
plt.title("Razón de secado vs tiempo")
plt.grid(True)
plt.show()


In [ ]:
#Ajuste de modelo
def newton_model(t, k):
    return np.exp(-k * t)

t_data = df["tiempo_min"].values
MR_data = df["MR"].values

# Ajuste de parámetros
popt_newton, pcov_newton = curve_fit(newton_model, t_data, MR_data, p0=[0.01])
k_newton = popt_newton[0]

print("Parámetro modelo de Newton:")
print(f"k = {k_newton:.5f}  (1/min)")


In [ ]:
#Modelo Page
def page_model(t, k, n):
    return np.exp(-k * t**n)

popt_page, pcov_page = curve_fit(page_model, t_data, MR_data, p0=[0.01, 1.0])
k_page, n_page = popt_page

print("Parámetros modelo de Page:")
print(f"k = {k_page:.5f}  (1/min^n)")
print(f"n = {n_page:.5f}")


In [ ]:
#comparar modelo con datos
t_fit = np.linspace(0, df["tiempo_min"].max(), 200)

MR_newton_fit = newton_model(t_fit, k_newton)
MR_page_fit = page_model(t_fit, k_page, n_page)

plt.figure(figsize=(6,4))
plt.scatter(t_data, MR_data, label="Datos experimentales", color="black")
plt.plot(t_fit, MR_newton_fit, label="Modelo Newton", linestyle="--")
plt.plot(t_fit, MR_page_fit, label="Modelo Page", linestyle="-.")

plt.xlabel("Tiempo (min)")
plt.ylabel("MR")
plt.title("Ajuste de modelos de secado")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#Métricas para evaluar los modelos
from sklearn.metrics import mean_squared_error

MR_newton_pred = newton_model(t_data, k_newton)
MR_page_pred = page_model(t_data, k_page, n_page)

rmse_newton = np.sqrt(mean_squared_error(MR_data, MR_newton_pred))
rmse_page = np.sqrt(mean_squared_error(MR_data, MR_page_pred))

print(f"RMSE Newton: {rmse_newton:.5f}")
print(f"RMSE Page:   {rmse_page:.5f}")


# 5. Covid-México

https://www.gob.mx/salud/documentos/datos-abiertos-bases-historicas-direccion-general-de-epidemiologia

In [ ]:
from google.colab import files

uploaded = files.upload()  # Ejecuta esto y selecciona el archivo .csv


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd

filename = "COVID19MEXICO.csv"  # 🔁 CAMBIA esto por el nombre real

# Primero intentamos con separador coma
try:
    df = pd.read_csv(filename, low_memory=False, encoding="latin1")
    print("Leído con separador por defecto (coma).")
except Exception as e:
    print("No se pudo leer con coma. Error:", e)
    print("Intentando con separador '|' ...")
    df = pd.read_csv(filename, sep="|", low_memory=False, encoding="latin1")
    print("Leído con separador '|'.")

# Ver primeras filas
df.head()


In [ ]:
print("Número de filas y columnas:", df.shape)
print("\nColumnas:")
print(df.columns.tolist())

df.info()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("COVID19MEXICO.csv", encoding="latin1", low_memory=False)
df.head()


In [ ]:
conteo_sexo = df["SEXO"].replace({1:"Mujer", 2:"Hombre"}).value_counts()
conteo_sexo.plot(kind="bar")
plt.title("Casos por sexo")
plt.xlabel("Sexo")
plt.ylabel("Número de casos")
plt.show()

In [ ]:
plt.hist(df["EDAD"], bins=40)
plt.title("Distribución de edades")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
comorbilidades = ["DIABETES", "OBESIDAD", "HIPERTENSION", "EPOC", "ASMA"]
conteos = df[comorbilidades].sum()

plt.figure(figsize=(8,4))
conteos.plot(kind="bar")
plt.title("Comorbilidades frecuentes")
plt.ylabel("Número de casos")
plt.show()


In [ ]:
df["TIPO_PACIENTE"].replace({1:"Ambulatorio", 2:"Hospitalizado"}).value_counts().plot(kind="bar")
plt.title("Tipo de paciente")
plt.ylabel("Número de casos")
plt.show()


In [ ]:
# Preparar los datos
tipo_paciente = df["TIPO_PACIENTE"].replace({1:"Ambulatorio", 2:"Hospitalizado"}).value_counts()

# Crear la figura con tamaño personalizado
plt.figure(figsize=(10, 6))

# Crear gráfico de barras con colores personalizados
colores = ['#2ecc71', '#e74c3c']  # Verde para ambulatorio, rojo para hospitalizado
barras = plt.bar(tipo_paciente.index, tipo_paciente.values, color=colores, alpha=0.8, edgecolor='black', linewidth=1.2)

# Agregar etiquetas con los valores encima de cada barra
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura,
             f'{int(altura):,}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Personalizar títulos y etiquetas
plt.title("Distribución de Casos por Tipo de Paciente", fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Tipo de Paciente", fontsize=12, fontweight='bold')
plt.ylabel("Número de Casos", fontsize=12, fontweight='bold')

# Mejorar la apariencia general
plt.xticks(fontsize=11)
plt.yticks(fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()

plt.show()

In [ ]:
# Convertir la columna FECHA_SINTOMAS a formato fecha
df["FECHA_SINTOMAS"] = pd.to_datetime(df["FECHA_SINTOMAS"], errors="coerce")



In [ ]:
casos_por_fecha = df.groupby("FECHA_SINTOMAS").size().sort_index()
casos_por_fecha.head()


In [ ]:
plt.figure(figsize=(14,5))
casos_por_fecha.plot()

plt.title("Casos nuevos por fecha de inicio de síntomas")
plt.xlabel("Fecha")
plt.ylabel("Número de casos")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(14,5))
casos_por_fecha.rolling(7).mean().plot()

plt.title("Casos por fecha (Media móvil de 7 días)")
plt.xlabel("Fecha")
plt.ylabel("Casos (promedio semanal)")
plt.grid(True)
plt.show()


In [ ]:
df["SEXO"] = df["SEXO"].replace({1: "Mujer", 2: "Hombre"})

casos_sexo = df.groupby(["FECHA_SINTOMAS", "SEXO"]).size().unstack()

plt.figure(figsize=(14,5))
casos_sexo.rolling(7).mean().plot()
plt.title("Casos por fecha y sexo (promedio móvil 7 días)")
plt.xlabel("Fecha")
plt.ylabel("Casos")
plt.show()


In [ ]:
#Correlación
variables = [
    "EDAD",
    "DIABETES",
    "OBESIDAD",
    "HIPERTENSION",
    "EPOC",
    "ASMA",
    "TABAQUISMO",
    "UCI",
    "INTUBADO",
    "TIPO_PACIENTE"
]


In [ ]:
df_corr = df[variables].copy()


In [ ]:
df_corr = df_corr.replace({97: np.nan, 98: np.nan, 99: np.nan})


In [ ]:
corr = df_corr.corr(method="spearman")
corr


In [ ]:
plt.figure(figsize=(10, 8),dpi=150)
sns.heatmap(corr, annot=True,
            cmap='coolwarm',
            linewidths=0.2,
            cbar_kws={'label': 'Coeficiente de Spearman'},
           vmin = -1, vmax = 1, center = 0)
plt.title('Matriz de Correlación', fontsize=12)
plt.xticks(rotation=45)
plt.show()

# 6. Exploración cuantitativa de textos literarios

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!pip install pdfplumber
import pdfplumber


In [ ]:
import pdfplumber #Extraer el texto del PDF

# Cambiar por el nombre del archivo subido
archivo_pdf = "continuidad.pdf"

texto = ""

with pdfplumber.open(archivo_pdf) as pdf:
    for pagina in pdf.pages:
        texto += pagina.extract_text() + " "


In [ ]:
texto[:500]


Limpiar el texto

pasar a minúsculas

quitar signos

eliminar números

eliminar saltos de línea

In [ ]:
import re

texto_limpio = texto.lower()
texto_limpio = re.sub(r"[^a-záéíóúñü\s]", "", texto_limpio)
texto_limpio = texto_limpio.replace("\n", " ")


In [ ]:
#Tokenizar (separar en palabras)
palabras = texto_limpio.split()
len(palabras) #Total de palabras

In [ ]:
#Frecuencia de palabras
from collections import Counter

frecuencias = Counter(palabras)
frecuencias.most_common(10)

In [ ]:
#Eliminar stopwords (palabras muy comunes)
stopwords = [
    "el", "la", "los", "las", "y", "de", "que", "en", "a", "un", "una",
    "se", "con", "por", "del", "al", "su", "sus", "lo"
]

palabras_filtradas = [p for p in palabras if p not in stopwords]

frecuencias_filtradas = Counter(palabras_filtradas)
frecuencias_filtradas.most_common(10)


In [ ]:
# import matplotlib.pyplot as plt

top = frecuencias_filtradas.most_common(10)

pal, frec = zip(*top)

plt.figure(figsize=(8,4))
plt.bar(pal, frec)
plt.xticks(rotation=45)
plt.title("Palabras más frecuentes")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
# Dividimos el texto en 3 partes
n = len(palabras_filtradas)
p1 = palabras_filtradas[0 : n//3]
p2 = palabras_filtradas[n//3 : 2*n//3]
p3 = palabras_filtradas[2*n//3 :]

Counter(p1).most_common(5), Counter(p2).most_common(5), Counter(p3).most_common(5)


# 7.Estabilidad de galletas funcionales

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error


In [ ]:
# Actividad de agua (ejemplo de 8 sales)
aw = np.array([0.11, 0.23, 0.33, 0.43, 0.53, 0.64, 0.75, 0.84])

# Parámetros "reales" para generar una isoterma tipo GAB (ejemplo)
Xm_true = 0.08   # humedad monomolecular (kg agua/kg sólidos secos)
C_true  = 10.0
k_true  = 0.90

def gab_model(aw, Xm, C, k):
    return (Xm * C * k * aw) / ((1 - k*aw) * (1 - k*aw + C*k*aw))

# Generamos X sintética con un poco de ruido
X_clean = gab_model(aw, Xm_true, C_true, k_true)
np.random.seed(42)
X = X_clean + np.random.normal(scale=0.003, size=len(aw))  # ruidecito pequeño

df = pd.DataFrame({"aw": aw, "X": X})
df


In [ ]:
def bet_model(aw, Xm, C):
    return (Xm * C * aw) / ((1 - aw) * (1 + (C - 1) * aw))

def gab_model(aw, Xm, C, k):
    return (Xm * C * k * aw) / ((1 - k*aw) * (1 - k*aw + C*k*aw))


In [ ]:
aw_data = df["aw"].values
X_data  = df["X"].values

# --- Ajuste BET ---
p0_bet = [0.05, 5.0]  # [Xm, C] valores iniciales
popt_bet, pcov_bet = curve_fit(bet_model, aw_data, X_data, p0=p0_bet)
Xm_bet, C_bet = popt_bet

# --- Ajuste GAB ---
p0_gab = [0.05, 5.0, 0.9]  # [Xm, C, k] valores iniciales
popt_gab, pcov_gab = curve_fit(gab_model, aw_data, X_data, p0=p0_gab, maxfev=10000)
Xm_gab, C_gab, k_gab = popt_gab

print("Parámetros BET:")
print(f"  Xm = {Xm_bet:.4f}")
print(f"  C  = {C_bet:.4f}\n")

print("Parámetros GAB:")
print(f"  Xm = {Xm_gab:.4f}")
print(f"  C  = {C_gab:.4f}")
print(f"  k  = {k_gab:.4f}")


In [ ]:
#Métricas
# Predicciones
X_bet_pred = bet_model(aw_data, Xm_bet, C_bet)
X_gab_pred = gab_model(aw_data, Xm_gab, C_gab, k_gab)

# RMSE
rmse_bet = np.sqrt(mean_squared_error(X_data, X_bet_pred))
rmse_gab = np.sqrt(mean_squared_error(X_data, X_gab_pred))

# R^2
ss_res_bet = np.sum((X_data - X_bet_pred)**2)
ss_res_gab = np.sum((X_data - X_gab_pred)**2)
ss_tot     = np.sum((X_data - np.mean(X_data))**2)

r2_bet = 1 - ss_res_bet/ss_tot
r2_gab = 1 - ss_res_gab/ss_tot

print("BET:  RMSE = {:.5f},  R² = {:.4f}".format(rmse_bet, r2_bet))
print("GAB:  RMSE = {:.5f},  R² = {:.4f}".format(rmse_gab, r2_gab))


In [ ]:
# Gráfica comparativa: DATOS + BET + GAB
# ------------------------------------------------------------

aw_fit = np.linspace(0.05, 0.90, 200)
X_bet_fit = bet_model(aw_fit, Xm_bet, C_bet)
X_gab_fit = gab_model(aw_fit, Xm_gab, C_gab, k_gab)

plt.figure(figsize=(7,5))
plt.scatter(aw_data, X_data, color="black", label="Datos experimentales")
plt.plot(aw_fit, X_bet_fit, label="Modelo BET ajustado")
plt.plot(aw_fit, X_gab_fit, label="Modelo GAB ajustado", linestyle="--")

plt.xlabel("Actividad de agua, $a_w$")
plt.ylabel("Humedad base seca, X (kg agua/kg ss)")
plt.title("Isoterma de sorción de galleta funcional")
plt.legend()
plt.grid(True)
plt.show()
